### Load Libraries

In [1]:
import pandas as pd
import numpy as np
import gc
import random
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm_notebook

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import xgboost as xgb
import lightgbm as lgb

### Load Train and Test data

In [2]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
traintest = pd.concat([train_df, test_df], axis = 0)

### Training data details

In [3]:
train_df.head()

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: float64(1845), int64(3147), object(1)
memory usage: 169.9+ MB


In [5]:
train_df.describe()

,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
count,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,...,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4459.000000,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03,4.459000e+03
mean,5.944923e+06,1.465493e+04,1.390895e+03,2.672245e+04,4.530164e+03,2.640996e+04,3.070811e+04,1.686522e+04,4.669208e+03,2.569407e+06,...,4.676057e+05,4.446239e+05,8.056219e+05,7.812966e+05,143.529939,1.213809e+05,3.573451e+04,3.123741e+05,9.219960e+04,2.279100e+05
std,8.234312e+06,3.893298e+05,6.428302e+04,5.699652e+05,2.359124e+05,1.514730e+06,5.770590e+05,7.512756e+05,1.879449e+05,9.610183e+06,...,4.068038e+06,4.428889e+06,4.513246e+06,6.839451e+06,9584.318507,4.720709e+06,1.614622e+06,4.318501e+06,1.635993e+06,1.811139e+06
min,3.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.000000e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.260000e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,8.000000e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.000000e+05,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,4.000000e+07,2.000000e+07,4.000000e+06,2.000000e+07,1.480000e+07,1.000000e+08,2.070800e+07,4.000000e+07,1.040000e+07,3.196120e+08,...,7.600000e+07,1.235880e+08,1.300000e+08,1.444000e+08,640000.000000,3.013120e+08,1.064200e+08,1.400000e+08,6.176800e+07,4.320000e+07


### Test Data Details

In [6]:
test_df.head()

,ID,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49342 entries, 0 to 49341
Columns: 4992 entries, ID to 9fc776466
dtypes: float64(4991), object(1)
memory usage: 1.8+ GB


In [8]:
test_df.describe()

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
count,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,...,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04,4.934200e+04
mean,5.773787e+04,6.258726e+04,1.036752e+05,6.289853e+04,6.713354e+04,8.083879e+04,6.181014e+04,5.515752e+04,1.406324e+06,8.128668e+04,...,1.193910e+05,1.355955e+05,3.242217e+05,1.437856e+05,9.302367e+04,8.047145e+04,6.076865e+04,1.323210e+05,1.675766e+05,1.282487e+05
std,1.745182e+06,2.322787e+06,2.586951e+06,2.765941e+06,3.206124e+06,2.845031e+06,2.780137e+06,1.923517e+06,6.872366e+06,2.378938e+06,...,3.115190e+06,2.598454e+06,3.782996e+06,3.663374e+06,5.041000e+06,2.100210e+06,2.040655e+06,3.592018e+06,3.761816e+06,2.413798e+06
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.504447e+08,2.283295e+08,2.758171e+08,3.972621e+08,4.667591e+08,2.852223e+08,4.863751e+08,2.043290e+08,3.435658e+08,2.310167e+08,...,5.351692e+08,1.236547e+08,3.793398e+08,4.025480e+08,9.657530e+08,1.680065e+08,2.497913e+08,3.200000e+08,3.186300e+08,2.189782e+08


### Check for missing values

In [9]:
train_df.columns[train_df.isnull().sum() != 0].size

0

In [10]:
test_df.columns[test_df.isnull().sum() != 0].size

0

### Remove Constant features

In [11]:
feats_counts = train_df.nunique(dropna = False)
const_cols = feats_counts.loc[feats_counts==1].index.tolist()

In [12]:
# remove constant features in the train and test set
train_df.drop(const_cols, axis=1, inplace=True)
test_df.drop(const_cols, axis=1, inplace=True) 

#print("Removed `{}` Constant Columns\n".format(len(const_cols)))
#print(const_cols)

### Remove Duplicated features

In [13]:
# Check and remove duplicate columns
colsToRemove = []
colsScaned = []
dupList = {}

columns = train_df.columns

for i in range(len(columns)-1):
    v = train_df[columns[i]].values
    dupCols = []
    for j in range(i+1,len(columns)):
        if np.array_equal(v, train_df[columns[j]].values):
            colsToRemove.append(columns[j])
            if columns[j] not in colsScaned:
                dupCols.append(columns[j]) 
                colsScaned.append(columns[j])
                dupList[columns[i]] = dupCols
                
# remove duplicate columns in the training set
train_df.drop(colsToRemove, axis=1, inplace=True) 

# remove duplicate columns in the testing set
test_df.drop(colsToRemove, axis=1, inplace=True)

print("Removed `{}` Duplicate Columns\n".format(len(dupList)))
print(dupList)

Removed `4` Duplicate Columns

{'34ceb0081': ['d60ddde1b'], '8d57e2749': ['acc5b709d', 'f333a5f60'], '168b3e5bc': ['f8d75792f'], 'a765da8bc': ['912836770']}


In [14]:
# saving them, as it takes long time to find them.

#import _pickle as pickle
#pickle.dump(dupList, open('dup_cols.p', 'wb'))

In [15]:
gc.collect()


7

### Preparing Data, Generating new features using aggregation and dimensionality reduction

In [16]:
X_train = train_df.drop(["ID", "target"], axis=1)
X_train_std = StandardScaler().fit_transform(X_train)
y_train = np.log1p(train_df["target"].values)

X_test = test_df.drop(["ID"], axis=1)
X_test_std = StandardScaler().fit_transform(X_test)

In [17]:
## Applying K-Means for clustering

for ncl in range(2,11):
    cls = KMeans(n_clusters=ncl)
    cls.fit_predict(X_train)
    X_train['kmeans_'+str(ncl)] = cls.predict(X_train)
    X_test['kmeans_'+str(ncl)] = cls.predict(X_test)

In [18]:
## New features using statistical Aggregation

X_train['Me_an']   = X_train.mean(axis=1)
X_train['Med_ian'] = X_train.median(axis=1)
X_train['Mo_de']   = X_train.mode(axis=1)
X_train['Ma_x']    = X_train.max(axis=1)
X_train['Va_r']    = X_train.var(axis=1)
X_train['St_d']    = X_train.std(axis=1)
        
X_test['Me_an']   = X_test.mean(axis=1)
X_test['Med_ian'] = X_test.median(axis=1)
X_test['Mo_de']   = X_test.mode(axis=1)
X_test['Ma_x']    = X_test.max(axis=1)
X_test['Va_r']    = X_test.var(axis=1)
X_test['St_d']    = X_test.std(axis=1)

In [19]:
#Get number of componenets for dimension reduction using pca
def _get_number_components(model, threshold):
    component_variance = model.explained_variance_ratio_
    explained_variance = 0.0
    components = 0

    for var in component_variance:
        explained_variance += var
        components += 1
        if(explained_variance >= threshold):
            break
    return components

### Get the optimal number of components
pca = PCA()
train_pca = pca.fit_transform(X_train_std)
components = _get_number_components(pca, threshold=0.90)

In [20]:
## Apply PCA

pca = PCA(n_components = components)
Xtrain_pca = pca.fit_transform(X_train_std)
Xtest_pca = pca.transform(X_test_std)

In [21]:
## Apply Truncated SVD 

tsvd = TruncatedSVD(n_components = components)
Xtrain_svd = tsvd.fit_transform(X_train_std)
Xtest_svd = tsvd.transform(X_test_std)

In [22]:
## Add the decomposed features (pca and svd) in the train & test dataset

def _add_decomposition(df, decomp, ncomp, flag):
    for i in range(1, ncomp+1):
        df[flag+"_"+str(i)] = decomp[:, i - 1]

_add_decomposition(X_train, Xtrain_pca, 1000, 'pca')
_add_decomposition(X_train, Xtrain_svd, 1000, 'svd')


_add_decomposition(X_test, Xtest_pca, 1000, 'pca')
_add_decomposition(X_test, Xtest_svd, 1000, 'svd')

In [23]:
## create the lists of decomposed and non decomposed features 

orig_features = [x for x in X_train.columns if "_" not in x]
decom_features = [x for x in X_train.columns if "_" in x]

## Split into train and validation data
dev_X, val_X, dev_y, val_y = train_test_split(X_train, y_train, test_size = 0.30, random_state = 42)

In [24]:
## Find important original features using Random Forests 

model1 = RandomForestRegressor(n_jobs=-1, random_state=42)
model1.fit(X_train[orig_features], y_train)
importances = model1.feature_importances_

## get list of important features 
orig_importances_df = pd.DataFrame({'importance': importances, 'feature': orig_features})
orig_importances_df = orig_importances_df.sort_values(by=['importance'], ascending=[False])
orig_important_features = orig_importances_df[:750]['feature'].values.tolist()

In [25]:
## Find important decomposed features using Random Forests 

model2 = RandomForestRegressor(n_jobs=-1, random_state=42)
model2.fit(X_train[decom_features], y_train)
importances = model2.feature_importances_

## get list of important features 
decom_importances_df = pd.DataFrame({'importance': importances, 'feature': decom_features})
decom_importances_df = decom_importances_df.sort_values(by=['importance'], ascending=[False])
decom_important_features = decom_importances_df[:750]['feature'].values.tolist()

In [26]:
## Group important features

important_features = orig_important_features + decom_important_features

## Training with LightGBM

In [27]:
## Create model with important features

def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 40,
        "learning_rate" : 0.005,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42
    }
    
    lgtrain = lgb.Dataset(train_X[important_features], label=train_y)
    lgval = lgb.Dataset(val_X[important_features], label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=100, 
                      evals_result=evals_result)
    
    pred_test_y = np.expm1(model.predict(test_X[important_features], num_iteration=model.best_iteration))
    return pred_test_y, model, evals_result

In [28]:
# Training LGB
pred_test_lgb, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, X_test)
print("LightGBM Training Completed...")

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.48623	valid_1's rmse: 1.57442
[200]	training's rmse: 1.29999	valid_1's rmse: 1.50909
[300]	training's rmse: 1.15956	valid_1's rmse: 1.47725
[400]	training's rmse: 1.04527	valid_1's rmse: 1.45853
[500]	training's rmse: 0.94783	valid_1's rmse: 1.44767
[600]	training's rmse: 0.86375	valid_1's rmse: 1.44139
[700]	training's rmse: 0.790464	valid_1's rmse: 1.43771
[800]	training's rmse: 0.72567	valid_1's rmse: 1.4359
[900]	training's rmse: 0.667261	valid_1's rmse: 1.43472
[1000]	training's rmse: 0.614677	valid_1's rmse: 1.43439
[1100]	training's rmse: 0.567213	valid_1's rmse: 1.43392
Early stopping, best iteration is:
[1099]	training's rmse: 0.567649	valid_1's rmse: 1.43391
LightGBM Training Completed...


## Training With XGBoost

In [29]:
def run_xgb(train_X, train_y, val_X, val_y, test_X):
    params = {'t',
          'eta': 0.005,
          'max_depth': 15, 
          'subsample': 0.7, 
          'colsample_bytree': 0.5,
          'alpha':0,
          'random_state': 42, 
          'silent': True}
    
    tr_data = xgb.DMatrix(train_X[important_features], train_y)
    va_data = xgb.DMatrix(val_X[important_features], val_y)
    
    watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
    model_xgb = xgb.train(params, tr_data, 2000, watchlist, maximize=False, early_stopping_rounds = 30, verbose_eval=100)
    
    dtest = xgb.DMatrix(test_X[important_features])
    xgb_pred_y = np.expm1(model_xgb.predict(dtest, ntree_limit=model_xgb.best_ntree_limit))
    
    return xgb_pred_y, model_xgb

In [30]:
# Training XGB
pred_test_xgb, model_xgb = run_xgb(dev_X, dev_y, val_X, val_y, X_test)
print("XGB Training Completed...")

[0]	train-rmse:14.0349	valid-rmse:14.0181
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 30 rounds.
[100]	train-rmse:8.60185	valid-rmse:8.587
[200]	train-rmse:5.3413	valid-rmse:5.36201
[300]	train-rmse:3.37948	valid-rmse:3.49674
[400]	train-rmse:2.17954	valid-rmse:2.4659
[500]	train-rmse:1.43047	valid-rmse:1.9266
[600]	train-rmse:0.955689	valid-rmse:1.66603
[700]	train-rmse:0.650107	valid-rmse:1.54635
[800]	train-rmse:0.450218	valid-rmse:1.49291
[900]	train-rmse:0.318528	valid-rmse:1.46836
[1000]	train-rmse:0.229327	valid-rmse:1.45683
[1100]	train-rmse:0.168969	valid-rmse:1.45166
[1200]	train-rmse:0.127006	valid-rmse:1.4489
[1300]	train-rmse:0.096771	valid-rmse:1.44729
[1400]	train-rmse:0.075007	valid-rmse:1.4465
[1500]	train-rmse:0.058802	valid-rmse:1.44599
[1600]	train-rmse:0.046728	valid-rmse:1.44565
[1700]	train-rmse:0.037689	valid-rmse:1.44543
[1800]	train-rmse:0.030641	valid-rmse:1.44535
[1900]

In [31]:
#pred_test = pred_test_xgb
pred_test = (pred_test_lgb + pred_test_xgb)/2

In [32]:
submission = pd.DataFrame({ 'ID': test_df.ID,
                            'target': pred_test})

In [33]:
submission.to_csv("submission.csv", index=False)